In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns

from os.path import join

plt.style.use(["seaborn", "thesis"])

/home/jo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
plt.rc("figure", figsize=(8,4))

# Fetch Dataset 

In [3]:
from SCFInitialGuess.utilities.dataset import extract_triu_batch, AbstractDataset
from sklearn.model_selection import train_test_split

data_path = "../../../dataset/TSmall_sto3g/"
postfix = "TSmall_sto3g"
dim = 26
#data_path = "../butadien/data/"
#postfix = ""
#dim = 26


def split(x, y, ind):
    return x[:ind], y[:ind], x[ind:], y[ind:]

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))
F = np.load(join(data_path, "F" + postfix + ".npy"))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))



ind = int(0.8 * len(molecules))
molecules_train, molecules_test = (molecules[:ind], molecules[ind:])

s_triu = extract_triu_batch(S, dim)
p_triu = extract_triu_batch(P, dim)

s_train, p_train, s_test, p_test = split(s_triu, p_triu, ind)

In [ ]:
model_path = "../../../models"

#  Calculate Descriptors

In [5]:
from SCFInitialGuess.descriptors.high_level import \
    AtomicNumberWeighted
from SCFInitialGuess.descriptors.coordinate_descriptors import \
    Gaussians, SPHAngularDescriptor
from SCFInitialGuess.descriptors.cutoffs import \
    BehlerCutoff1
from SCFInitialGuess.descriptors.models import \
    RADIAL_GAUSSIAN_MODELS, make_uniform
import pickle    

    
descriptor = AtomicNumberWeighted(
    #Gaussians(*make_uniform(40, 80, 20)),
    Gaussians(*RADIAL_GAUSSIAN_MODELS["Man"]),
    SPHAngularDescriptor(4),
    BehlerCutoff1(5)
)

pickle.dump(descriptor, open(model_path + "descriptor.dump", "wb"))

print(descriptor.number_of_descriptors)

100


In [6]:
descriptor.calculate_all_descriptors(molecules[0]).shape

(10, 100)

In [7]:
G = []
for mol in molecules:
    G.append(
        descriptor.calculate_all_descriptors(mol).flatten()
    )
    
G = np.asarray(G)

In [8]:
g_train, g_test = G[:ind], G[ind:]

In [9]:
G.shape

(1001, 1000)

# Setup Networks 

In [10]:
dim_in = G.shape[1]
dim_triu = dim * (dim + 1) // 2

print(dim**2, dim_triu, dim_in)

676 351 1000


In [11]:
keras.backend.clear_session()


activation = "elu"
learning_rate = 1e-5
intializer = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01)

model = keras.Sequential()

# input layer
model.add(keras.layers.Dense(700, activation=activation, input_dim=dim_in, kernel_initializer=intializer, bias_initializer='zeros'))

# hidden
#for i in range(3):

model.add(keras.layers.Dense(
    500, 
    activation=activation, 
    kernel_initializer=intializer, 
    #bias_initializer='zeros',
    #kernel_regularizer=keras.regularizers.l2(1e-8)
))


model.add(keras.layers.Dense(
        400, 
        activation=activation, 
        kernel_initializer=intializer, 
        #bias_initializer='zeros',
        #kernel_regularizer=keras.regularizers.l2(1e-8)
))


#output
model.add(keras.layers.Dense(dim_triu))

model.compile(optimizer=keras.optimizers.Adam(learning_rate), loss='MSE', metrics=['mae', 'mse'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 700)               700700    
_________________________________________________________________
dense_1 (Dense)              (None, 500)               350500    
_________________________________________________________________
dense_2 (Dense)              (None, 400)               200400    
_________________________________________________________________
dense_3 (Dense)              (None, 351)               140751    
Total params: 1,392,351
Trainable params: 1,392,351
Non-trainable params: 0
_________________________________________________________________


In [12]:
filepath = "../../../models/model_descriptos_" + postfix + ".h5"

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_mean_squared_error", 
    min_delta=1e-7, 
    patience=200, 
    verbose=1
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=50, 
    verbose=1, 
    mode='auto', 
    min_delta=1e-4, 
    cooldown=50, 
    min_lr=1e-10
)

checkpoint = keras.callbacks.ModelCheckpoint(
    filepath, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=False, 
    save_weights_only=False, 
    mode='auto', 
    period=1
)



In [1]:
i=0

In [13]:
i+=1
log_dir = "./log/Butadien_" + str(i)
tensorboard = keras.callbacks.TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    batch_size=32, 
    #update_freq='epoch'
)

# Do the training 

In [14]:
g_train.shape, p_train.shape

((800, 1000), (800, 351))

In [ ]:
epochs = 10000

while True:
    keras.backend.set_value(model.optimizer.lr, 1e-4)

    history = model.fit(
        x = g_train,
        y = p_train,
        epochs=epochs,
        shuffle=True,
        validation_data=(g_test, p_test), 
        verbose=1, 
        callbacks=[
            early_stopping, 
            reduce_lr, 
            checkpoint
        ]
    )
    
    print("\n\n\n\n\nNext rouuund!!")

Train on 800 samples, validate on 201 samples
Epoch 1/10000
800/800 [==============================] - 0s 583us/step - loss: 0.0012 - mean_absolute_error: 0.0198 - mean_squared_error: 0.0012 - val_loss: 5.7059e-04 - val_mean_absolute_error: 0.0173 - val_mean_squared_error: 5.7059e-04

Epoch 00001: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 2/10000
800/800 [==============================] - 0s 527us/step - loss: 2.6259e-04 - mean_absolute_error: 0.0117 - mean_squared_error: 2.6259e-04 - val_loss: 1.2807e-04 - val_mean_absolute_error: 0.0083 - val_mean_squared_error: 1.2807e-04

Epoch 00002: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 3/10000
800/800 [==============================] - 0s 552us/step - loss: 7.1623e-05 - mean_absolute_error: 0.0062 - mean_squared_error: 7.1623e-05 - val_loss: 6.0133e-05 - val_mean_absolute_error: 0.0057 - val_mean_squared_error: 6.0133e-05

Epoch 00003: saving model to ../../../models/model_descriptos_

Epoch 26/10000
800/800 [==============================] - 0s 430us/step - loss: 1.0473e-05 - mean_absolute_error: 0.0025 - mean_squared_error: 1.0473e-05 - val_loss: 2.4198e-05 - val_mean_absolute_error: 0.0036 - val_mean_squared_error: 2.4198e-05

Epoch 00026: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 27/10000
800/800 [==============================] - 0s 404us/step - loss: 1.0417e-05 - mean_absolute_error: 0.0025 - mean_squared_error: 1.0417e-05 - val_loss: 2.4209e-05 - val_mean_absolute_error: 0.0036 - val_mean_squared_error: 2.4209e-05

Epoch 00027: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 28/10000
800/800 [==============================] - 0s 391us/step - loss: 1.0202e-05 - mean_absolute_error: 0.0025 - mean_squared_error: 1.0202e-05 - val_loss: 2.3925e-05 - val_mean_absolute_error: 0.0036 - val_mean_squared_error: 2.3925e-05

Epoch 00028: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 29/10000
800/

Epoch 76/10000
800/800 [==============================] - 0s 552us/step - loss: 6.5564e-06 - mean_absolute_error: 0.0020 - mean_squared_error: 6.5564e-06 - val_loss: 1.9044e-05 - val_mean_absolute_error: 0.0032 - val_mean_squared_error: 1.9044e-05

Epoch 00076: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 77/10000
800/800 [==============================] - 0s 556us/step - loss: 6.6179e-06 - mean_absolute_error: 0.0020 - mean_squared_error: 6.6179e-06 - val_loss: 1.8906e-05 - val_mean_absolute_error: 0.0032 - val_mean_squared_error: 1.8906e-05

Epoch 00077: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 78/10000
800/800 [==============================] - 0s 532us/step - loss: 6.5259e-06 - mean_absolute_error: 0.0020 - mean_squared_error: 6.5259e-06 - val_loss: 1.8954e-05 - val_mean_absolute_error: 0.0032 - val_mean_squared_error: 1.8954e-05

Epoch 00078: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 79/10000
800/

Epoch 126/10000
800/800 [==============================] - 1s 1ms/step - loss: 6.4161e-06 - mean_absolute_error: 0.0020 - mean_squared_error: 6.4161e-06 - val_loss: 1.7897e-05 - val_mean_absolute_error: 0.0031 - val_mean_squared_error: 1.7897e-05

Epoch 00126: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 127/10000
800/800 [==============================] - 1s 1ms/step - loss: 6.4565e-06 - mean_absolute_error: 0.0020 - mean_squared_error: 6.4565e-06 - val_loss: 1.8318e-05 - val_mean_absolute_error: 0.0031 - val_mean_squared_error: 1.8318e-05

Epoch 00127: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 128/10000
800/800 [==============================] - 1s 1ms/step - loss: 6.5676e-06 - mean_absolute_error: 0.0020 - mean_squared_error: 6.5676e-06 - val_loss: 1.7926e-05 - val_mean_absolute_error: 0.0031 - val_mean_squared_error: 1.7926e-05

Epoch 00128: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 129/10000
800/80

Epoch 176/10000
800/800 [==============================] - 1s 1ms/step - loss: 4.0562e-06 - mean_absolute_error: 0.0016 - mean_squared_error: 4.0562e-06 - val_loss: 1.4999e-05 - val_mean_absolute_error: 0.0028 - val_mean_squared_error: 1.4999e-05

Epoch 00176: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 177/10000
800/800 [==============================] - 1s 1ms/step - loss: 4.0807e-06 - mean_absolute_error: 0.0016 - mean_squared_error: 4.0807e-06 - val_loss: 1.4938e-05 - val_mean_absolute_error: 0.0028 - val_mean_squared_error: 1.4938e-05

Epoch 00177: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 178/10000
800/800 [==============================] - 1s 1ms/step - loss: 4.0177e-06 - mean_absolute_error: 0.0016 - mean_squared_error: 4.0177e-06 - val_loss: 1.4865e-05 - val_mean_absolute_error: 0.0028 - val_mean_squared_error: 1.4865e-05

Epoch 00178: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 179/10000
800/80

...

Epoch 751/10000
800/800 [==============================] - 1s 829us/step - loss: 1.9184e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.9184e-06 - val_loss: 1.1315e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1315e-05

Epoch 00751: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 752/10000
800/800 [==============================] - 1s 698us/step - loss: 1.9183e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.9183e-06 - val_loss: 1.1317e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1317e-05

Epoch 00752: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 753/10000
800/800 [==============================] - 1s 675us/step - loss: 1.9179e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.9179e-06 - val_loss: 1.1309e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1309e-05

Epoch 00753: ReduceLROnPlateau reducing learning rate to 3.906249901319825e-07.

Epoch 00753: 

Epoch 776/10000
800/800 [==============================] - 1s 1ms/step - loss: 1.8998e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.8998e-06 - val_loss: 1.1302e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1302e-05

Epoch 00776: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 777/10000
800/800 [==============================] - 1s 1ms/step - loss: 1.8995e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.8995e-06 - val_loss: 1.1302e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1302e-05

Epoch 00777: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 778/10000
800/800 [==============================] - 1s 1ms/step - loss: 1.9001e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.9001e-06 - val_loss: 1.1300e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1300e-05

Epoch 00778: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 779/10000
800/80

800/800 [==============================] - 1s 1ms/step - loss: 1.8940e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.8940e-06 - val_loss: 1.1281e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1281e-05

Epoch 00826: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 827/10000
800/800 [==============================] - 1s 1ms/step - loss: 1.8946e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.8946e-06 - val_loss: 1.1289e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1289e-05

Epoch 00827: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 828/10000
800/800 [==============================] - 1s 1ms/step - loss: 1.8945e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.8945e-06 - val_loss: 1.1284e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1284e-05

Epoch 00828: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 829/10000
800/800 [=============

Epoch 876/10000
800/800 [==============================] - 1s 1ms/step - loss: 1.8823e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.8823e-06 - val_loss: 1.1273e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1273e-05

Epoch 00876: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 877/10000
800/800 [==============================] - 1s 858us/step - loss: 1.8816e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.8816e-06 - val_loss: 1.1272e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1272e-05

Epoch 00877: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 878/10000
800/800 [==============================] - 1s 734us/step - loss: 1.8817e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.8817e-06 - val_loss: 1.1273e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1273e-05

Epoch 00878: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 879/10000
80

800/800 [==============================] - 1s 1ms/step - loss: 1.8793e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.8793e-06 - val_loss: 1.1266e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1266e-05

Epoch 00926: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 927/10000
800/800 [==============================] - 1s 1ms/step - loss: 1.8785e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.8785e-06 - val_loss: 1.1265e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1265e-05

Epoch 00927: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 928/10000
800/800 [==============================] - 1s 1ms/step - loss: 1.8791e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 1.8791e-06 - val_loss: 1.1267e-05 - val_mean_absolute_error: 0.0024 - val_mean_squared_error: 1.1267e-05

Epoch 00928: saving model to ../../../models/model_descriptos_TSmall_sto3g.h5
Epoch 929/10000
800/800 [=============